In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
import copy 
import logging

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [3]:
def get_information(method, params):
#     url = 'https://api.s0.os.hmny.io/'
    url = 'https://api.s0.dry.hmny.io'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [6]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    return block, last_block

In [7]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [8]:
def getAdjustment():
    method = 'hmy_getCurrentUtilityMetrics'
    params = []
    num = get_information(method, params)['result']['Adjustment']
    return float(num)

In [9]:
def getBlockSigners(blockNum):
    method = 'hmy_getBlockSigners'
    params = [blockNum]
    return get_information(method, params)['result']

In [13]:
def R6_test(single):
    logger.info(f"Test-R6: Tests whether the delegation reward is distributed correctly")
    
    block, last_block = getCurrentAndLastBlock()
    logger.info(f"current and last block numbers: {block}, {last_block}")

    if block != last_block:
        while block < last_block:
            block = getBlockNumber()
        logger.info(f"current block {block}, will begin collecting delegator infos...")
        
    
    acc_rewards_prev = dict()
    delegations_prev = dict()
    total_stake = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            acc_rewards_prev[address] = reward_accumulated
            ds = i['validator']['delegations']
            dels = dict()
            for d in ds:
                d_addr = d['delegator-address']
                d_reward = d['reward']
                dels[d_addr] = d_reward
            delegations_prev[address] = dels
            total_stake[address] = i['total-delegation']
    logger.info(f"wait until the 20th block to wait for non-beacon shard delay")
    next_block = block + 19
    while block < next_block:
        block = getBlockNumber()
    flag = True
    num = 2
    iterations = 0
    while iterations < num:
        next_block = block + 1
        while block < next_block:
            block = getBlockNumber()
        logger.info(f"new block {block} reached, will begin testing...")
        # get the validator info and compute validator rewards
        acc_rewards_curr = dict()
        delegations_curr = dict()
        validator_infos = getAllValidatorInformation()
        for i in validator_infos:
            if i['currently-in-committee'] == True:
                address = i['validator']['address']
                if (address not in delegations_prev) or (address not in acc_rewards_prev):
                    continue
                reward_accumulated = i['lifetime']['reward-accumulated']
                acc_rewards_curr[address] = reward_accumulated
                reward = reward_accumulated - acc_rewards_prev[address]
                if reward == 0:
                    continue
                commission = float(i['validator']['rate']) * reward
                total_delegation_reward = reward - commission
                total_delegation = total_stake[address]
                ds = i['validator']['delegations']            
                dels = delegations_prev[address]
                dels_curr = dict()
                for d in ds:
                    d_addr = d['delegator-address']
                    if d_addr not in dels:
                        continue
                    d_reward = d['reward']
                    dels_curr[d_addr] = d_reward
                    d_amount = d['amount']
                    delegation_reward_actual = d_reward
                    if d_addr in dels:
                        delegation_reward_actual = delegation_reward_actual - dels[d_addr]
                    percentage = d_amount / total_delegation
                    delegation_reward_expected = percentage * total_delegation_reward
                    if d_addr == address:
                        delegation_reward_expected = delegation_reward_expected + commission
                    if format(delegation_reward_actual, '.15e') != format(delegation_reward_expected, '.15e'):
                        logger.warning(f"Test-R6: Fail")
                        logger.warning(f"for validator {address} delegation {d_addr}, expected: {delegation_reward_expected:.15e}, actual: {delegation_reward_actual:.15e}\n")
                        flag = False
                delegations_curr[address] = dels_curr
        iterations += 1
        acc_rewards_prev = copy.deepcopy(acc_rewards_curr)
        delegations_prev = copy.deepcopy(delegations_curr)
    if single:
        curr_test = None
    else:
        curr_test = R7_test
    if flag:
        logger.info(f"Test-R6: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [14]:
R6_test(True)

INFO:economic-test:Test-R6: Tests whether the delegation reward is distributed correctly
INFO:economic-test:current and last block numbers: 3225157, 3225169


current and last block numbers 3225157 3225169


INFO:economic-test:current block 3225169, will begin collecting delegator infos...
INFO:economic-test:wait until the 20th block to wait for non-beacon shard delay
INFO:economic-test:new block 3225189 reached, will begin testing...




INFO:economic-test:new block 3225190 reached, will begin testing...



(False, None)